In [270]:
import ccxt
from datetime import datetime
import pandas as pd
from sympy import Limit
import numpy as np



## Fetch Ohlcv

In [281]:


def fetchData(exchange, symbol, timeframe, since="26-04-2022", limit=30):
    exchange_list = [
        "kraken",
        'ftx',
        'currencycom',
        'coinmate' ]
    try:
        exchange = getattr(ccxt, exchange)()
    except AttributeError:
        print("-" * 36, " ERROR ", "-" * 35)
        print(
            'Exchange "{}" not found. Please check the exchange is supported.'.format(
                exchange
            )
        )
        print("Supported exchanges are:")
        print(exchange_list)
        print("-" * 80)
        quit()

    data = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
    header = ["Timestamp", "open", "high", "low", "close", "volume"]
    df = pd.DataFrame(data, columns=header)

    df.Timestamp = (
        df.Timestamp / 1000
    ) 
    df["date"] = pd.to_datetime(df.Timestamp, unit="s")

    df = df[["date", "open", "high", "low", "close", "volume"]]

    df["open"] = pd.to_numeric(df["open"])
    df["high"] = pd.to_numeric(df["high"])
    df["low"] = pd.to_numeric(df["low"])
    df["close"] = pd.to_numeric(df["close"])
    df["volume"] = pd.to_numeric(df["volume"])

    return df

In [282]:
print(fetchData("kraken",'BTC/USDT',"1m",30))

                  date     open     high      low    close    volume
0  2022-04-28 00:47:00  39319.4  39319.4  39319.4  39319.4  0.013214
1  2022-04-28 00:48:00  39319.4  39350.8  39319.4  39350.8  0.250173
2  2022-04-28 00:49:00  39367.9  39372.9  39367.9  39372.9  0.010198
3  2022-04-28 00:50:00  39375.0  39442.8  39375.0  39385.4  0.095567
4  2022-04-28 00:51:00  39385.4  39385.4  39385.4  39385.4  0.000000
5  2022-04-28 00:52:00  39338.4  39338.4  39323.4  39323.4  0.004313
6  2022-04-28 00:53:00  39323.4  39323.4  39323.4  39323.4  0.000000
7  2022-04-28 00:54:00  39346.0  39346.0  39346.0  39346.0  0.002470
8  2022-04-28 00:55:00  39346.0  39346.0  39346.0  39346.0  0.000000
9  2022-04-28 00:56:00  39338.6  39338.6  39338.6  39338.6  0.003200
10 2022-04-28 00:57:00  39338.6  39338.6  39338.6  39338.6  0.000000
11 2022-04-28 00:58:00  39365.3  39365.3  39365.3  39365.3  0.003175
12 2022-04-28 00:59:00  39370.5  39370.5  39370.5  39370.5  0.003094
13 2022-04-28 01:00:00  39394.3  3

## Funciones-Fecthc Order Book para los distintos apis

In [261]:
def fetch_order_book_kraken(symbol,limit):
    exchange = ccxt.kraken()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfk = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    dfk["Total Volume"]=(dfk.AVolume + dfk.BVolume)
    dfk["Midprice"]=((dfk.Ask + dfk.Bid)/2)
    dfk['Vwap Ask'] =(np.cumsum(dfk.Ask*dfk.AVolume) / np.cumsum(dfk.AVolume))
    dfk['Vwap Bid'] =(np.cumsum(dfk.Bid*dfk.BVolume) / np.cumsum(dfk.BVolume))
    
                       
    return dfk


In [ ]:
def fetch_order_book_ftx(symbol,limit):
    exchange = ccxt.ftx()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dff = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dff['Vwap Ask'] =(np.cumsum(dff.Ask*dff.AVolume) / np.cumsum(dff.AVolume))
    dff['Vwap Bid'] =(np.cumsum(dff.Bid*dff.BVolume) / np.cumsum(dff.BVolume))
    dff["Total Volume"]=(dff.AVolume + dff.BVolume)
    dff["Midprice"]=((dff.Ask + dff.Bid)/2)
    dff['Vwap Ask'] =(np.cumsum(dff.Ask*df.AVolume) / np.cumsum(dff.AVolume))
    dff['Vwap Bid'] =(np.cumsum(dff.Bid*df.BVolume) / np.cumsum(dff.BVolume))
    
                       
    return dff


In [262]:
def fetch_order_book_currencycom(symbol,limit):
    exchange = ccxt.currencycom()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfc = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfc['Vwap Ask'] =(np.cumsum(dfc.Ask*dfc.AVolume) / np.cumsum(dfc.AVolume))
    dfc['Vwap Bid'] =(np.cumsum(dfc.Bid*dfc.BVolume) / np.cumsum(dfc.BVolume))
    dfc["Total Volume"]=(dfc.AVolume + dfc.BVolume)
    dfc["Midprice"]=((dfc.Ask + dfc.Bid)/2)
    dfc['Vwap Ask'] =(np.cumsum(dfc.Ask*dfc.AVolume) / np.cumsum(dfc.AVolume))
    dfc['Vwap Bid'] =(np.cumsum(dfc.Bid*dfc.BVolume) / np.cumsum(dfc.BVolume))
    
                       
    return dfc

In [ ]:
def fetch_order_book_coinmate(symbol,limit):
    exchange = ccxt.coinmate()
    bi_btc_ob = exchange.fetch_order_book(symbol,limit=limit)
    bi_btc_ob_ask=pd.DataFrame(bi_btc_ob["asks"],columns=["Ask","AVolume"])
    bi_btc_ob_bid=pd.DataFrame(bi_btc_ob["bids"],columns=["Bid","BVolume"])
    dfcm = pd.concat([bi_btc_ob_ask,  bi_btc_ob_bid],axis=1)
    dfcm['Vwap Ask'] =(np.cumsum(dfcm.Ask*dfcm.AVolume) / np.cumsum(dfcm.AVolume))
    dfcm['Vwap Bid'] =(np.cumsum(dfcm.Bid*dfcm.BVolume) / np.cumsum(dfcm.BVolume))
    dfcm["Total Volume"]=(dfcm.AVolume + dfcm.BVolume)
    dfcm["Midprice"]=((dfcm.Ask + dfcm.Bid)/2)
    dfcm['Vwap Ask'] =(np.cumsum(dfcm.Ask*dfcm.AVolume) / np.cumsum(dfcm.AVolume))
    dfcm['Vwap Bid'] =(np.cumsum(dfcm.Bid*dfcm.BVolume) / np.cumsum(dfcm.BVolume))
    
                       
    return dfcm